# Importation des packages

In [57]:
%load_ext autoreload
%autoreload 2 
%reload_ext autoreload

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as mtick
from matplotlib.colors import ListedColormap
from pathlib import Path
from pprint import pprint
import seaborn as sns
import pickle
import json
import xgboost

import sys
import os
import gc
# Le chemin vers le dossier packages
packages_path = os.path.join(os.path.dirname(os.getcwd()), 'packages')
# Ajouter le chemin du dossier packages à sys.path
sys.path.append(packages_path)
import warnings

# Filtrer les avertissements spécifiques
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message="ntree_limit is deprecated")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Configuration des paths

In [2]:

# Configuration des chemins de base
DATA_PATH = 'G:/DCCR/DPP/MAIA/DSA/1_Leasing/04_Octroi/Backtesting/202503_Backtesting_sim/DATA/'
MODELS_PATH = 'G:/DCCR/DPP/MAIA/DSA/1_Leasing/04_Octroi/Backtesting/202503_Backtesting_sim/MODELS/'

# Chemin bases necessaires que pour BQ
BASE_DEMANDE_PATH = 'G:/DCCR/DPP/MAIA/DSA/1_Leasing/00_Bases/SAS/'

# Chemins des modèles par perimetre
MODEL_PERIM_PATHS = {
    'BQ': f'{MODELS_PATH}BQ/',
    'ST': f'{MODELS_PATH}ST/',
    'HST': f'{MODELS_PATH}HST/'
}

# Chemins des données par perimetre
DATA_PERIM_PATHS = {
    'BQ': f'{DATA_PATH}BQ/',
    'ST': f'{DATA_PATH}ST/',
    'HST': f'{DATA_PATH}HST/'
}

# # Exemple d'usage
# bq_data_path = DATA_PERIM_PATHS['BQ']
# st_model_path = MODEL_PERIM_PATHS['ST']

#### Import du modèle et de son processing

In [3]:
def load_model_components(model_key, MODEL_PATH):
    """
    Charge les composants d'un modèle spécifique
    Args:
        model_key: 'BQ', 'ST' ou 'HST'
    Returns:
        dict: {'model': XGBoost model, 'features': list, 'pipeline': sklearn pipeline, 'classifier': object}
    """
       
    model_path = MODEL_PATH[model_key]
    
    # 1. Charger le modèle XGBoost (format JSON)
    model = xgboost.XGBClassifier()
    model.load_model(os.path.join(model_path, 'model.json'))
    
    # 2. Charger les features (format pickle)
    with open(os.path.join(model_path, 'features.pkl'), 'rb') as f:
        features = pickle.load(f)
    
    # 3. Charger le pipeline (format pickle)
    with open(os.path.join(model_path, 'pipe.pkl'), 'rb') as f:
        pipeline = pickle.load(f)
    
    # 4. Charger le classifieur de risque (format pickle)
    with open(os.path.join(model_path, 'classifier.pkl'), 'rb') as f:
        classifier = pickle.load(f)
    
    return {
        'model': model,
        'features': features,
        'pipeline': pipeline,
        'classifier': classifier
    }

In [4]:
bq_components = load_model_components('BQ', MODEL_PERIM_PATHS)
st_components = load_model_components('ST', MODEL_PERIM_PATHS)
hst_components = load_model_components('HST', MODEL_PERIM_PATHS)

In [5]:
# Pour le modèle BQ
bq_model = bq_components['model']
bq_features = bq_components['features']
bq_pipeline = bq_components['pipeline']
bq_scorer = bq_components['classifier']

In [6]:
# Pour le modèle ST
st_model = st_components['model']
st_features = st_components['features']
st_pipeline = st_components['pipeline']
st_scorer = st_components['classifier']

In [7]:
# Pour le modèle HST
hst_model = hst_components['model']
hst_features = hst_components['features']
hst_pipeline = hst_components['pipeline']
hst_scorer = hst_components['classifier']

### Import des bases

BASE ORIGIN

In [8]:
# Configuration des chemins

SAVE_PATHS = {
    'BQ': f"{DATA_PERIM_PATHS['BQ']}base_defaut_bq_data_prep.csv",
    'ST': f"{DATA_PERIM_PATHS['ST']}base_defaut_st_data_prep.csv",
    'HST': f"{DATA_PERIM_PATHS['HST']}base_defaut_hst_data_prep.csv"
}

# Réimportation des bases
for perim, path in SAVE_PATHS.items():
    # Vérification que le fichier existe
    if os.path.exists(path):
        # Lecture du fichier CSV
        df = pd.read_csv(path)
        
        # Conversion de la date si nécessaire
        if 'DATEDEMANDE' in df.columns:
            df['DATEDEMANDE'] = pd.to_datetime(df['DATEDEMANDE'])
        
        # Assignation à une variable globale avec le même nom
        globals()[f'base_defaut_{perim.lower()}'] = df
        
        # Vérification
        print(f"\n=== Réimportation {perim} ===")
        print(f"Dimensions : {df.shape}")
        if 'DATEDEMANDE' in df.columns:
            print("Plage de dates DATEDEMANDE :")
            print(df['DATEDEMANDE'].agg(["min", "max"]))
        print(f"Importé depuis : {path}")
        print('-'*60)
    else:
        print(f"\n!!! Fichier non trouvé pour {perim} : {path} !!!")

print("\n*** Réimportation terminée pour tous les périmètres ***")

C:\Users\untesucl\AppData\Local\Temp\ipykernel_48880\673396502.py:14: DtypeWarning: Columns (90,93,441) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)



=== Réimportation BQ ===
Dimensions : (161316, 464)
Plage de dates DATEDEMANDE :
min   2014-01-30
max   2025-02-25
Name: DATEDEMANDE, dtype: datetime64[ns]
Importé depuis : G:/DCCR/DPP/MAIA/DSA/1_Leasing/04_Octroi/Backtesting/202503_Backtesting_sim/DATA/BQ/base_defaut_bq_data_prep.csv
------------------------------------------------------------


C:\Users\untesucl\AppData\Local\Temp\ipykernel_48880\673396502.py:14: DtypeWarning: Columns (66,93,259) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)



=== Réimportation ST ===
Dimensions : (183522, 430)
Plage de dates DATEDEMANDE :
min   2014-03-24
max   2025-02-26
Name: DATEDEMANDE, dtype: datetime64[ns]
Importé depuis : G:/DCCR/DPP/MAIA/DSA/1_Leasing/04_Octroi/Backtesting/202503_Backtesting_sim/DATA/ST/base_defaut_st_data_prep.csv
------------------------------------------------------------


C:\Users\untesucl\AppData\Local\Temp\ipykernel_48880\673396502.py:14: DtypeWarning: Columns (19,90,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)



=== Réimportation HST ===
Dimensions : (59567, 430)
Plage de dates DATEDEMANDE :
min   2014-03-19
max   2025-02-19
Name: DATEDEMANDE, dtype: datetime64[ns]
Importé depuis : G:/DCCR/DPP/MAIA/DSA/1_Leasing/04_Octroi/Backtesting/202503_Backtesting_sim/DATA/HST/base_defaut_hst_data_prep.csv
------------------------------------------------------------

*** Réimportation terminée pour tous les périmètres ***


REF

In [9]:
def load_reference_samples(model_type, files):
    """
    Charge l'échantillon de référence pour un type de modèle donné avec reporting détaillé.
    
    Args:
        model_type: Le type de modèle ('BQ', 'ST', 'HST')
        files: Dictionnaire de configuration des fichiers
    
    Returns:
        DataFrame ou None en cas d'erreur
    """
    try:
        filename = files[model_type]['train']
        filepath = Path(MODEL_PERIM_PATHS[model_type]) / filename
        
        # Chargement des données
        sample = pd.read_pickle(filepath)
        col_date= 'DT_MEL_MIN'
        
        # Affichage des informations détaillées
        print(f"\n=== Chargement {model_type} ===")
        print(f"Chemin d'import : {filepath}")
        print(f"Dimensions : {sample.shape}")
        print(f"\nPlage de dates {col_date} :")
        print(sample[col_date].agg(["min", "max"]))
      
        
        print(f"\n✅ {model_type}: {len(sample)} lignes chargées")
        print('-'*60)
        
        return sample
    
    except FileNotFoundError:
        print(f"\n⚠️ Fichier non trouvé: {filepath}")
        return None
    except Exception as e:
        print(f"\n⚠️ Erreur lors du chargement ({model_type}): {str(e)}")
        return None

In [10]:
# Configuration des noms de fichiers
SAMPLE_FILES = {
    'BQ': {'train': 'train_set_vf.pkl'},
    'ST': {'train': 'train_set.pkl'},
    'HST': {'train': 'train_set.pkl'} 
} 

# Chargement de tous les échantillons avec option d'inclusion de validation pour les analyses de performance
reference_samples = {
    model_type: load_reference_samples(model_type, SAMPLE_FILES)
    for model_type in ['BQ', 'ST', 'HST']
}
# Echantillons de reference par perimetre 
ref_sample_bq = reference_samples['BQ']
ref_sample_st = reference_samples['ST']
ref_sample_hst = reference_samples['HST']


=== Chargement BQ ===
Chemin d'import : G:\DCCR\DPP\MAIA\DSA\1_Leasing\04_Octroi\Backtesting\202503_Backtesting_sim\MODELS\BQ\train_set_vf.pkl
Dimensions : (27387, 418)

Plage de dates DT_MEL_MIN :
min   2017-04-01
max   2019-03-31
Name: DT_MEL_MIN, dtype: datetime64[ns]

✅ BQ: 27387 lignes chargées
------------------------------------------------------------

=== Chargement ST ===
Chemin d'import : G:\DCCR\DPP\MAIA\DSA\1_Leasing\04_Octroi\Backtesting\202503_Backtesting_sim\MODELS\ST\train_set.pkl
Dimensions : (31845, 412)

Plage de dates DT_MEL_MIN :
min   2017-04-01
max   2019-03-31
Name: DT_MEL_MIN, dtype: datetime64[ns]

✅ ST: 31845 lignes chargées
------------------------------------------------------------

=== Chargement HST ===
Chemin d'import : G:\DCCR\DPP\MAIA\DSA\1_Leasing\04_Octroi\Backtesting\202503_Backtesting_sim\MODELS\HST\train_set.pkl
Dimensions : (9677, 412)

Plage de dates DT_MEL_MIN :
min   2017-04-01
max   2019-03-31
Name: DT_MEL_MIN, dtype: datetime64[ns]

✅ HST

BKT

In [11]:
# Configuration des chemins

SAVE_PATHS_BKT = {
    'BQ': f"{DATA_PERIM_PATHS['BQ']}bkt_sample_bq.csv",
    'ST': f"{DATA_PERIM_PATHS['ST']}bkt_sample_st.csv",
    'HST': f"{DATA_PERIM_PATHS['HST']}bkt_sample_hst.csv"
}

# Réimportation des bases
for perim, path in SAVE_PATHS_BKT.items():
    # Vérification que le fichier existe
    if os.path.exists(path):
        # Lecture du fichier CSV
        df = pd.read_csv(path)
        
        # Conversion des dates si nécessaire
        if 'DT_MEL_MIN' in df.columns:
            df['DT_MEL_MIN'] = pd.to_datetime(df['DT_MEL_MIN'])
        
        # Assignation à une variable globale avec le même nom
        globals()[f'bkt_sample_{perim.lower()}'] = df
        
        # Vérification
        print(f"\n=== Réimportation {perim} ===")
        print(f"Dimensions : {df.shape}")
        if 'DT_MEL_MIN' in df.columns:
            print("Plage de dates DT_MEL_MIN :")
            print(df['DT_MEL_MIN'].agg(["min", "max"]))
        print(f"Importé depuis : {path}")
        print('-'*60)
    else:
        print(f"\n!!! Fichier non trouvé pour {perim} : {path} !!!")

print("\n*** Réimportation terminée pour tous les périmètres ***")

C:\Users\untesucl\AppData\Local\Temp\ipykernel_48880\3161472603.py:14: DtypeWarning: Columns (90,93,441) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)



=== Réimportation BQ ===
Dimensions : (65986, 464)
Plage de dates DT_MEL_MIN :
min   2020-01-01
max   2023-06-30
Name: DT_MEL_MIN, dtype: datetime64[ns]
Importé depuis : G:/DCCR/DPP/MAIA/DSA/1_Leasing/04_Octroi/Backtesting/202503_Backtesting_sim/DATA/BQ/bkt_sample_bq.csv
------------------------------------------------------------


C:\Users\untesucl\AppData\Local\Temp\ipykernel_48880\3161472603.py:14: DtypeWarning: Columns (93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)



=== Réimportation ST ===
Dimensions : (74513, 430)
Plage de dates DT_MEL_MIN :
min   2020-01-01
max   2023-06-30
Name: DT_MEL_MIN, dtype: datetime64[ns]
Importé depuis : G:/DCCR/DPP/MAIA/DSA/1_Leasing/04_Octroi/Backtesting/202503_Backtesting_sim/DATA/ST/bkt_sample_st.csv
------------------------------------------------------------

=== Réimportation HST ===
Dimensions : (21108, 430)
Plage de dates DT_MEL_MIN :
min   2020-01-01
max   2023-06-30
Name: DT_MEL_MIN, dtype: datetime64[ns]
Importé depuis : G:/DCCR/DPP/MAIA/DSA/1_Leasing/04_Octroi/Backtesting/202503_Backtesting_sim/DATA/HST/bkt_sample_hst.csv
------------------------------------------------------------

*** Réimportation terminée pour tous les périmètres ***


C:\Users\untesucl\AppData\Local\Temp\ipykernel_48880\3161472603.py:14: DtypeWarning: Columns (93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


### Filtre base origin pour obtenir le périmètre de train pour comparer les references. 

In [54]:
base_defaut_bq['DT_MEL_MIN'].agg(["min", "max"])

min    2015-02-01
max    2025-02-28
Name: DT_MEL_MIN, dtype: object

Ce filtre est nécessaire pour obtenir la base train pour tous les modèles à partir de la base par défaut qui couvre la période février 2015 - février 2025. Ces bases nous serviront de comparaison avec les périodes de référence récupérées au moment de la modélisation (ref_sample_xx), et nous aideront également à amener ces références à la forme à laquelle nous pouvons appliquer le pipeline de traitement qui existe maintenant en production et qui nécessite certaines variables d'entrée, qui ne se trouvent pas actuellement dans ces références.

### BQ

In [12]:
ref_sample_bq.DT_MEL_MIN.min(), ref_sample_bq.DT_MEL_MIN.max(), ref_sample_bq.shape,

(Timestamp('2017-04-01 00:00:00'),
 Timestamp('2019-03-31 00:00:00'),
 (27387, 418))

In [13]:
def filtrer_dataframe(df, date_col='DT_MEL_MIN'):
    return df[(df[date_col] >= '2017-04-01') & (df[date_col] <= '2019-03-31')]

base_defaut_train_bq = filtrer_dataframe(base_defaut_bq)
base_defaut_train_bq.shape, base_defaut_train_bq['DT_MEL_MIN'].agg(["min", "max"])

((29154, 464),
 min    2017-04-01
 max    2019-03-31
 Name: DT_MEL_MIN, dtype: object)

Je veux savoir d'ou il y a ces differences entre les 2 qui normalement sont censées à avoir le même nombre des lignes.

In [14]:
## 1. Vérification des doublons dans les clés
print("Doublons dans ref_sample_bq:", ref_sample_bq.duplicated(subset=['NUMERODOSSIER', 'NUMERODEMANDE']).sum())
print("Doublons dans base_defaut_train_bq:", base_defaut_train_bq.duplicated(subset=['NUMERODOSSIER', 'NUMERODEMANDE']).sum())

Doublons dans ref_sample_bq: 0
Doublons dans base_defaut_train_bq: 6


In [15]:
# Suppression des doublons sur NUM_DEC et TYPO pour 2024
base_defaut_train_bq = base_defaut_train_bq.drop_duplicates(subset=["NUMERODOSSIER", "NUMERODEMANDE"], keep='first')
print(f"Nombre de lignes après la suppression doublons: {len(base_defaut_train_bq)}")

Nombre de lignes après la suppression doublons: 29148


In [16]:
## 2. Identification des éléments uniques à chaque DataFrame
# Ceux dans base_defaut_train_bq mais pas dans ref_sample_bq
unique_to_base = base_defaut_train_bq[
    ~base_defaut_train_bq.set_index(['NUMERODOSSIER', 'NUMERODEMANDE']).index.isin(
        ref_sample_bq.set_index(['NUMERODOSSIER', 'NUMERODEMANDE']).index
    )
]
unique_to_base

,NUM_DEC,DT_MEL_MIN,MAX_DEFAUT_BTR,D_ENTRE_DEF,MAX_HORIZON,ACTION,CODEGRILLE,NUMEROBTCTP,SIRENCTP,CODETYPEPRODFINANCIER,...,nb_n,accep_n,transforme_n,YEAR,NUMERODOSSIER,NUMERODEMANDE,flag_accord_gles,RET_CORP,top_gles_1,Flag_GreenLease
33,200044-J0,2018-11-01,0.0,999999999.0,13.0,SAIN,B1,M001675272,788040160,CESS,...,1.0,1.0,1.0,2019.0,200044-J0,2.0,NR,RETAIL,score,GreenLease
40,200048-J0,2018-12-01,0.0,999999999.0,12.0,SAIN,B1,M001690745,531464410,CESS,...,1.0,1.0,1.0,2019.0,200048-J0,1.0,NR,RETAIL,score,GreenLease
45,200056-H0,2017-06-20,0.0,999999999.0,18.0,SAIN,B1,M001645712,495083339,STAN,...,1.0,1.0,1.0,2017.0,200056-H0,1.0,NR,RETAIL,score,Hors GreenLease
49,200060-I0,2018-03-14,0.0,999999999.0,19.0,SAIN,B1,M001509855,538605353,PROF,...,1.0,1.0,1.0,2018.0,200060-I0,1.0,O,RETAIL,non,GreenLease Hors Périmètre
127,200139-I0,2018-01-11,0.0,999999999.0,83.0,SAIN,B1,I001182022,500617733,STAN,...,1.0,1.0,1.0,2018.0,200139-I0,2.0,NR,RETAIL,score,GreenLease
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136726,372821-I0,2019-01-22,0.0,999999999.0,11.0,SAIN,B1,M001730632,809351406,PROF,...,1.0,1.0,1.0,2018.0,372821-I0,1.0,O,RETAIL,non,GreenLease Hors Périmètre
136818,372997-I0,2018-11-01,0.0,999999999.0,12.0,SAIN,B1,M001584024,509702064,CESS,...,1.0,1.0,1.0,2018.0,372997-I0,2.0,NR,RETAIL,score,GreenLease
136836,373026-I0,2019-01-28,0.0,999999999.0,18.0,SAIN,B1,M001456477,793419102,PROF,...,1.0,1.0,1.0,2018.0,373026-I0,1.0,O,CORPORATE,non,GreenLease Hors Périmètre
136994,373317-I0,2019-03-10,0.0,999999999.0,11.0,SAIN,B1,M001431840,340467935,CESS,...,1.0,1.0,1.0,2019.0,373317-I0,2.0,NR,RETAIL,score,GreenLease


In [17]:
unique_to_base.NUMERODEMANDE.value_counts(dropna=False), unique_to_base.NUMERODOSSIER.value_counts(dropna=False)

(1.0    1737
 2.0     503
 NaN       1
 Name: NUMERODEMANDE, dtype: int64,
 200044-J0    1
 308631-I0    1
 308220-H0    1
 308252-H0    1
 308257-I0    1
             ..
 249883-H0    1
 249686-I0    1
 249422-I0    1
 249335-I0    1
 373353-I0    1
 Name: NUMERODOSSIER, Length: 2241, dtype: int64)

In [18]:
# Ceux dans ref_sample_bq mais pas dans base_defaut_train_bq
unique_to_ref = ref_sample_bq[
    ~ref_sample_bq.set_index(['NUMERODOSSIER', 'NUMERODEMANDE']).index.isin(
        base_defaut_train_bq.set_index(['NUMERODOSSIER', 'NUMERODEMANDE']).index
    )
]
unique_to_ref

,NUM_DEC,DT_MEL_MIN,MAX_DEFAUT_BTR,D_ENTRE_DEF,MAX_HORIZON,ACTION,CODEGRILLE,NUMEROBTCTP,SIRENCTP,CODETYPEPRODFINANCIER,...,IMMOCORPORELLES,ACTIFSIMMOBILIERS,CREDITBAILLOCATION,EBIT,RESOPEAPRESELEMENTFIAVANTIMPOT,DISPONIBLESTOTAL,INDICATEURETATLIASSE,MESSAGEERREURETATLIASSE,MONTH_DATE_FINANCEMENT,QUARTER_DATE_FINANCEMENT
18,200139-I0,2018-01-11,0.0,999999999.0,45.0,SAIN,B1,I001182022,500617733,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01,2018-Q1
106,200536-I0,2018-01-23,0.0,999999999.0,38.0,SAIN,B1,M001133473,502333925,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01,2018-Q1
121,200578-H0,2017-06-01,0.0,999999999.0,52.0,SAIN,B1,M001352608,073201212,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-06,2017-Q2
150,200713-H0,2017-07-31,0.0,999999999.0,44.0,SAIN,B1,M001646020,438945289,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07,2017-Q3
152,200719-I0,2018-02-01,0.0,999999999.0,35.0,SAIN,B1,M001124808,389979113,CESS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02,2018-Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27221,371774-I0,2019-01-29,0.0,999999999.0,33.0,SAIN,B1,M000498571,350905816,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01,2019-Q1
27253,372100-I0,2019-01-11,0.0,999999999.0,26.0,SAIN,B1,M001066128,444742209,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01,2019-Q1
27268,372234-I0,2019-01-14,0.0,999999999.0,33.0,SAIN,B1,M001750319,801475823,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01,2019-Q1
27292,372483-I0,2018-12-01,0.0,999999999.0,26.0,SAIN,B1,M001101353,433497617,CESS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12,2018-Q4


In [19]:
unique_to_ref.NUMERODEMANDE.value_counts(dropna=False), unique_to_ref.NUMERODOSSIER.value_counts(dropna=False)

(1.0    480
 Name: NUMERODEMANDE, dtype: int64,
 200139-I0    1
 200536-I0    1
 301025-H0    1
 299899-H0    1
 299286-I0    1
             ..
 243144-I0    1
 243132-I0    1
 242887-H0    1
 242851-J0    1
 373353-I0    1
 Name: NUMERODOSSIER, Length: 480, dtype: int64)

In [20]:
##  Vérification des dates des éléments uniques
if not unique_to_base.empty:
    print("\nDates des éléments uniques à base_defaut_train_bq:")
    print(unique_to_base['DT_MEL_MIN'].agg(["min", "max"]))
    
if not unique_to_ref.empty:
    print("\nDates des éléments uniques à ref_sample_bq:")
    print(unique_to_ref['DT_MEL_MIN'].agg(["min", "max"]))


Dates des éléments uniques à base_defaut_train_bq:
min    2017-04-01
max    2019-03-29
Name: DT_MEL_MIN, dtype: object

Dates des éléments uniques à ref_sample_bq:
min   2017-04-01
max   2019-03-28
Name: DT_MEL_MIN, dtype: datetime64[ns]


### ST

In [21]:
base_defaut_train_st = filtrer_dataframe(base_defaut_st)
base_defaut_train_st.shape, base_defaut_train_st['DT_MEL_MIN'].agg(["min", "max"])

((33078, 430),
 min    2017-04-01
 max    2019-03-31
 Name: DT_MEL_MIN, dtype: object)

In [22]:
ref_sample_st.shape, ref_sample_st['DT_MEL_MIN'].agg(["min", "max"])

((31845, 412),
 min   2017-04-01
 max   2019-03-31
 Name: DT_MEL_MIN, dtype: datetime64[ns])

In [23]:
## 1. Vérification des doublons dans les clés
print("Doublons dans ref_sample_st:", ref_sample_st.duplicated(subset=['NUMERODOSSIER', 'NUMERODEMANDE']).sum())
print("Doublons dans base_defaut_train_st:", base_defaut_train_st.duplicated(subset=['NUMERODOSSIER', 'NUMERODEMANDE']).sum())

Doublons dans ref_sample_st: 0
Doublons dans base_defaut_train_st: 0


In [24]:
## 2. Identification des éléments uniques à chaque DataFrame
# Ceux dans base_defaut_train_st mais pas dans ref_sample_st
unique_to_base_st = base_defaut_train_st[
    ~base_defaut_train_st.set_index(['NUMERODOSSIER', 'NUMERODEMANDE']).index.isin(
        ref_sample_st.set_index(['NUMERODOSSIER', 'NUMERODEMANDE']).index
    )
]
unique_to_base_st

,NUM_DEC,DT_MEL_MIN,MAX_DEFAUT_BTR,D_ENTRE_DEF,MAX_HORIZON,ACTION,CODEGRILLE,NUMEROBTCTP,SIRENCTP,CODETYPEPRODFINANCIER,...,D_STATUT,GR1,GR2,NUMEROBTPARTENAIRE,DATE_ARRETE_BDF,NOMCTP,DATEIMMATCTP,DATE_ARRETE_SRR,TYPO,RET_CORP
197,200231-I0,2018-04-01,0.0,999999999.0,7.0,SAIN,S1,M000304749,556350049,LMAN,...,2018-01-03 15:08:28,6- SCORE,6- Decision Score,M000273360,2017-10-19,SM3,1963-01-01,NaN,AUTRE,CORPORATE
238,200276-I0,2018-01-01,0.0,999999999.0,11.0,SAIN,S1,M001078172,490223179,LMAF,...,2018-01-03 15:55:17,6- SCORE,6- Decision Score,M001074159,2017-12-18,DABSTART,2006-06-01,NaN,AUTRE,RETAIL
313,200364-I0,2018-04-10,0.0,999999999.0,17.0,SAIN,S1,I001622815,789365095,LMAF,...,2018-01-03 17:50:33,6- SCORE,6- Decision Score,M000001072,2018-01-03,BESSUAND,2012-11-01,NaN,AUTRE,RETAIL
417,200494-I0,2018-07-01,0.0,999999999.0,10.0,SAIN,S1,M000514456,385277793,LMAF,...,2018-01-04 11:33:14,6- SCORE,6- Decision Score,M000021354,2018-01-04,PROFESSION SPORT,1992-03-23,2017-12-11,AUTRE,RETAIL
504,200588-H0,2017-04-01,0.0,999999999.0,13.0,SAIN,S1,M000503918,377793062,STAN,...,2017-01-04 16:52:55,6- SCORE,6- Decision Score,M000273360,2017-01-04,TRANSPORTS DE SAVOIE,1990-04-20,NaN,AUTRE,CORPORATE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157038,369455-I0,2019-01-01,0.0,999999999.0,18.0,SAIN,S1,M000326882,349816603,AAR,...,2018-12-19 09:46:19,6- SCORE,6- Decision Score,M000355961,2018-12-19,TDP - IP TELEPHONIE DU PILAT,1989-03-01,2018-12-06,AUTRE,RETAIL
157667,370269-I0,2019-01-01,0.0,999999999.0,15.0,SAIN,S1,M001732262,841412513,LMAF,...,2018-12-21 10:16:43,3- DRE LE MANS,3- DRE Le Mans - Gp II,M000290985,2018-09-03,AI2P,2018-07-30,NaN,AUTRE,RETAIL
158434,371471-I0,2019-01-01,0.0,999999999.0,11.0,SAIN,S1,M000305530,419683818,LMAN,...,2018-12-21 12:23:53,6- SCORE,6- Decision Score,M000273360,2018-07-30,IVECO FRANCE,1998-07-23,NaN,AUTRE,CORPORATE
158896,372198-I0,2019-01-01,0.0,999999999.0,1.0,SAIN,S1,M000613786,428748909,LMAN,...,2018-12-26 11:54:38,6- SCORE,6- Decision Score,M000273360,2018-09-25,SOC ETUDES REALIS GESTION IMMOB CONSTRUC,1999-12-27,NaN,AUTRE,CORPORATE


In [25]:
# Ceux dans ref_sample_st mais pas dans base_defaut_train_st
unique_to_ref_st = ref_sample_st[
    ~ref_sample_st.set_index(['NUMERODOSSIER', 'NUMERODEMANDE']).index.isin(
        base_defaut_train_st.set_index(['NUMERODOSSIER', 'NUMERODEMANDE']).index
    )
]
unique_to_ref_st

,NUM_DEC,DT_MEL_MIN,MAX_DEFAUT_BTR,D_ENTRE_DEF,MAX_HORIZON,ACTION,CODEGRILLE,NUMEROBTCTP,SIRENCTP,CODETYPEPRODFINANCIER,...,ACTIFSIMMOBILIERS,CREDITBAILLOCATION,EBIT,RESOPEAPRESELEMENTFIAVANTIMPOT,DISPONIBLESTOTAL,INDICATEURETATLIASSE,MESSAGEERREURETATLIASSE,DT_MEL2,MONTH_DATE_FINANCEMENT,QUARTER_DATE_FINANCEMENT
3534,215603-J0,2018-04-01,0.0,999999999.0,38.0,SAIN,H1,M000575011,885650473,CESS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-04-01,2018-04,2018-Q2
31844,317599-I0,2018-09-26,0.0,999999999.0,35.0,SAIN,H1,M001735655,510546476,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-26,2018-09,2018-Q3


### HST

In [26]:
base_defaut_train_hst = filtrer_dataframe(base_defaut_hst)
base_defaut_train_hst.shape, base_defaut_train_hst['DT_MEL_MIN'].agg(["min", "max"])

((10028, 430),
 min    2017-04-01
 max    2019-03-31
 Name: DT_MEL_MIN, dtype: object)

In [27]:
ref_sample_hst.shape, ref_sample_hst['DT_MEL_MIN'].agg(["min", "max"])

((9677, 412),
 min   2017-04-01
 max   2019-03-31
 Name: DT_MEL_MIN, dtype: datetime64[ns])

In [28]:
## 1. Vérification des doublons dans les clés
print("Doublons dans ref_sample_hst:", ref_sample_hst.duplicated(subset=['NUMERODOSSIER', 'NUMERODEMANDE']).sum())
print("Doublons dans base_defaut_train_hst:", base_defaut_train_hst.duplicated(subset=['NUMERODOSSIER', 'NUMERODEMANDE']).sum())

Doublons dans ref_sample_hst: 0
Doublons dans base_defaut_train_hst: 0


In [29]:
## 2. Identification des éléments uniques à chaque DataFrame
# Ceux dans base_defaut_train_hst mais pas dans ref_sample_hst
unique_to_base_hst = base_defaut_train_hst[
    ~base_defaut_train_hst.set_index(['NUMERODOSSIER', 'NUMERODEMANDE']).index.isin(
        ref_sample_hst.set_index(['NUMERODOSSIER', 'NUMERODEMANDE']).index
    )
]
unique_to_base_hst

,NUM_DEC,DT_MEL_MIN,MAX_DEFAUT_BTR,D_ENTRE_DEF,MAX_HORIZON,ACTION,CODEGRILLE,NUMEROBTCTP,SIRENCTP,CODETYPEPRODFINANCIER,...,D_STATUT,GR1,GR2,NUMEROBTPARTENAIRE,DATE_ARRETE_BDF,NOMCTP,DATEIMMATCTP,DATE_ARRETE_SRR,TYPO,RET_CORP
60,200170-I0,2018-05-01,0.0,999999999.0,16.0,SAIN,H1,I001500875,330505694,CESS,...,2018-01-03 14:05:40,6- SCORE,6- Decision Score,M001060483,2018-01-03,DESMARCHELIER,1984-02-20,2017-12-11,AUTRE,RETAIL
101,200273-I0,2018-04-01,0.0,999999999.0,19.0,SAIN,H1,M001461669,530500776,CESS,...,2018-01-05 09:36:19,3- DRE LE MANS,3- DRE Le Mans - Gp II,M000353413,2017-11-30,C-LOG SOLUTIONS,2011-02-22,NaN,AUTRE,CORPORATE
288,200923-I0,2018-04-01,0.0,999999999.0,1.0,SAIN,H1,M001274218,484378575,LMAF,...,2018-01-05 11:43:53,6- SCORE,6- Decision Score,M000356134,2018-01-05,L'IMMOBILIERE WEISS,2005-10-01,2017-12-11,AUTRE,RETAIL
583,201752-J0,2019-01-28,0.0,999999999.0,18.0,SAIN,H1,M001128783,502218183,STAN,...,2019-01-08 18:25:46,4- DDR,4- DR Siege,M000286580,2018-12-18,COUDERT AUTOMOBILES,2008-02-01,2019-01-07,AUTRE,RETAIL
635,201917-H0,2017-05-01,0.0,999999999.0,13.0,SAIN,H1,M000654955,315334730,STAN,...,2017-01-10 07:35:41,6- SCORE,6- Decision Score,M000273360,2017-01-10,IMMOBILIERE DU DOUAISIS,1979-01-01,NaN,AUTRE,RETAIL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49112,360809-I0,2018-12-19,0.0,999999999.0,14.0,SAIN,H1,M001212164,499133049,STAN,...,2019-01-17 14:21:39,4- DDR,4- DR SE,M000286580,2018-12-04,MENUISIER COMPAGNONS - BRUNO PRADIER,2007-07-17,2018-11-08,AUTRE,RETAIL
49870,364342-I0,2019-02-01,0.0,999999999.0,10.0,SAIN,H1,M000576614,964503304,CESS,...,2018-12-21 16:13:34,3- DRE LE MANS,3- DRE Le Mans - Gp II,M001122945,2018-12-10,ELECTRICITE GENERALE APPLIQUEE,1964-01-01,NaN,AUTRE,CORPORATE
50216,365969-I0,2019-01-01,0.0,999999999.0,9.0,SAIN,H1,M001129095,431865450,STAN,...,2018-12-12 17:34:02,6- SCORE,6- Decision Score,M000273360,2018-06-11,ELENBI,2000-05-09,NaN,AUTRE,CORPORATE
50537,367569-I0,2019-02-01,0.0,999999999.0,13.0,SAIN,H1,M001545304,387471485,CESS,...,2018-12-18 18:03:21,3- DRE LE MANS,3- DRE Le Mans - Autres,M001122945,2018-12-14,NOUVEAU MONDE,1992-04-06,NaN,AUTRE,CORPORATE


In [30]:
# Ceux dans ref_sample_hst mais pas dans base_defaut_train_hst
unique_to_ref_hst = ref_sample_hst[
    ~ref_sample_hst.set_index(['NUMERODOSSIER', 'NUMERODEMANDE']).index.isin(
        base_defaut_train_hst.set_index(['NUMERODOSSIER', 'NUMERODEMANDE']).index
    )
]
unique_to_ref_hst

,NUM_DEC,DT_MEL_MIN,MAX_DEFAUT_BTR,D_ENTRE_DEF,MAX_HORIZON,ACTION,CODEGRILLE,NUMEROBTCTP,SIRENCTP,CODETYPEPRODFINANCIER,...,ACTIFSIMMOBILIERS,CREDITBAILLOCATION,EBIT,RESOPEAPRESELEMENTFIAVANTIMPOT,DISPONIBLESTOTAL,INDICATEURETATLIASSE,MESSAGEERREURETATLIASSE,DT_MEL2,MONTH_DATE_FINANCEMENT,QUARTER_DATE_FINANCEMENT
1056,216756-F0,2018-10-31,0.0,999999999.0,36.0,SAIN,H1,M001536104,505025247,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-10-31,2018-10,2018-Q4
9243,354059-I0,2019-01-01,0.0,999999999.0,28.0,SAIN,H1,M001666064,447758541,LMAF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01,2019-01,2019-Q1
9599,211643-J0,2019-03-27,0.0,999999999.0,31.0,SAIN,H1,M000452270,310590864,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-27,2019-03,2019-Q1
9600,265788-I0,2018-06-28,0.0,999999999.0,40.0,SAIN,H1,M000672863,876950387,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-06-28,2018-06,2018-Q2
9601,265898-I0,2018-07-01,0.0,999999999.0,39.0,SAIN,H1,M001717832,824400436,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-07-01,2018-07,2018-Q3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9672,361744-I0,2019-01-07,0.0,999999999.0,33.0,SAIN,H1,M000293681,403554181,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-07,2019-01,2019-Q1
9673,361971-I0,2019-01-28,0.0,999999999.0,33.0,SAIN,H1,M000305968,338543770,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-28,2019-01,2019-Q1
9674,363746-I0,2019-02-19,0.0,999999999.0,32.0,SAIN,H1,M001182609,492494513,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-02-19,2019-02,2019-Q1
9675,363985-I0,2019-01-01,0.0,999999999.0,33.0,SAIN,H1,M001352926,534312004,STAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01,2019-01,2019-Q1


In [31]:
def print_model_comparison(model_name, ref_train, bkt_data):
    """Affiche un tableau comparatif simplifié entre train et backtesting"""
    print(f"\n{'='*50}")
    print(f"COMPARAISON {model_name.upper():^20}")
    print(f"{'='*50}")
    print(f"{'MÉTRIQUE':<35} | {'TRAIN (REF)':>14} | {'TRAIN (BASE DEFAUT)':>14}")
    print(f"{'-'*35}-|{'-'*16}|{'-'*14}")
    
    # Calcul des métriques
    metrics = [
        ("Taux de cible par échantillon (%)", 
         ref_train['CIBLE_20M'].mean()*100,
         bkt_data['CIBLE_20M'].mean()*100),
        
        ("Volume par échantillon (K)", 
         ref_train.shape[0]/1000,
         bkt_data.shape[0]/1000),
        
        ("Part du volume par échantillon (%)", 
        round(ref_train.shape[0]/(ref_train.shape[0]+bkt_data.shape[0])*100,1),
        round(bkt_data.shape[0]/(bkt_data.shape[0]+ref_train.shape[0])*100,1)),

        ("Part de défaut en valeur (cible=1)",  # Part de défaut en valeur
         ref_train['CIBLE_20M'].sum(),
         bkt_data['CIBLE_20M'].sum())
    ]

    
    # Affichage formaté
    for name, ref_val, bkt_val in metrics:
        print(f"{name:<35} | {ref_val:>14.2f} | {bkt_val:>14.2f}")

In [32]:
models_data = {
    'BQ': (ref_sample_bq, base_defaut_train_bq),
    'ST': (ref_sample_st, base_defaut_train_st),
    'HST': (ref_sample_hst, base_defaut_train_hst)
}

# Affichage pour chaque modèle
for model_name, (ref_data, bkt_data) in models_data.items():
    print_model_comparison(model_name, ref_data, bkt_data)


COMPARAISON          BQ         
MÉTRIQUE                            |    TRAIN (REF) | TRAIN (BASE DEFAUT)
------------------------------------|----------------|--------------
Taux de cible par échantillon (%)   |           2.55 |           2.53
Volume par échantillon (K)          |          27.39 |          29.15
Part du volume par échantillon (%)  |          48.40 |          51.60
Part de défaut en valeur (cible=1)  |         698.00 |         736.00

COMPARAISON          ST         
MÉTRIQUE                            |    TRAIN (REF) | TRAIN (BASE DEFAUT)
------------------------------------|----------------|--------------
Taux de cible par échantillon (%)   |           3.41 |           3.52
Volume par échantillon (K)          |          31.84 |          33.08
Part du volume par échantillon (%)  |          49.10 |          50.90
Part de défaut en valeur (cible=1)  |        1086.00 |        1163.00

COMPARAISON         HST         
MÉTRIQUE                            |    TRAIN (RE

## Analyse pipeline de processing

Des anomalies ont été identifiées dans les pipelines de traitement. Avant d'appliquer les corrections, nous allons analyser ces anomalies et évaluer l'impact des correctifs. Le véritable effet sera mesuré lors du calcul des indicateurs de stabilité.

Avant d'appliquer les pipelines, il faut faire un mapping pour que les données qu'on retrouve dans nos bases correspondent au traitement applique par la pipeline. 
Ce mapping est necessaire car  il y a une difference entre ce qu'on reçoit en termes des données de DE ( comme ça se passe en prod) et l'equivalent qu'on a dans nos bases. 

### Data cleaner

Il y a plusieurs choses qui sont necessaires à ajouter au data cleaner.

In [55]:
bq_pipeline.named_steps.keys(), st_pipeline.named_steps.keys(), hst_pipeline.named_steps.keys()

(dict_keys(['data_cleaner', 'quali_pipe']),
 dict_keys(['data_cleaner', 'quali_pipe']),
 dict_keys(['data_cleaner', 'quali_pipe']))

In [66]:
steps = bq_pipeline.named_steps.keys()
steps

dict_keys(['data_cleaner', 'quali_pipe'])

In [67]:
steps['quali_pipe'].named_steps['COTECREDIT'].dict_feature_values

TypeError: 'dict_keys' object is not subscriptable

In [62]:
from sklearn.utils import estimator_html_repr
print(estimator_html_repr(bq_pipeline))

<style>#sk-c4049c59-c61e-4594-a8db-10751667e389 {color: black;background-color: white;}#sk-c4049c59-c61e-4594-a8db-10751667e389 pre{padding: 0;}#sk-c4049c59-c61e-4594-a8db-10751667e389 div.sk-toggleable {background-color: white;}#sk-c4049c59-c61e-4594-a8db-10751667e389 label.sk-toggleable__label {cursor: pointer;display: block;width: 100%;margin-bottom: 0;padding: 0.3em;box-sizing: border-box;text-align: center;}#sk-c4049c59-c61e-4594-a8db-10751667e389 label.sk-toggleable__label-arrow:before {content: "▸";float: left;margin-right: 0.25em;color: #696969;}#sk-c4049c59-c61e-4594-a8db-10751667e389 label.sk-toggleable__label-arrow:hover:before {color: black;}#sk-c4049c59-c61e-4594-a8db-10751667e389 div.sk-estimator:hover label.sk-toggleable__label-arrow:before {color: black;}#sk-c4049c59-c61e-4594-a8db-10751667e389 div.sk-toggleable__content {max-height: 0;max-width: 0;overflow: hidden;text-align: left;background-color: #f0f8ff;}#sk-c4049c59-c61e-4594-a8db-10751667e389 div.sk-toggleable__co

### 1. S'assurer qu'on a toutes les colonnes necessaires pour le traitement du data_cleaner

In [33]:
## liste necessaire pour data_cleaner

features_necessaires = {
   "NOTEBDFCTP": "Note Banque de France de la contrepartie",
   "DATESITUATIONRISQPIRECTP": "Date de la situation de risque pire de la contrepartie",
   "DATEDERNIERREFUSCTP": "Date du dernier refus de la contrepartie",
   "DATEDERNIERACCORDCTP": "Date du dernier accord de la contrepartie",
   "DATECLOTUREBILAN": "Date de clôture du bilan",
   "DATEIMMATCTP": "Date d'immatriculation de la contrepartie",
   "DATE_ARRETE_NOR": "Date d'arrêté de la note NOR",
   "DATE_ARRETE_NOFC": "Date d'arrêté de la note NOFC",
   "DATE_ARRETE_SRR": "Date d'arrêté de la note SRR",
   "DATEDEMANDE": "Date de la demande",
   "PAYDEX": "Indicateur de paiement",
   "EXPOPOTENTIELLEBRUTEGROUPE": "Exposition potentielle brute du groupe",
   "DETTESINF1AN": "Dettes inférieures à 1 an",
   "EXIGIBLEATERME": "Exigible à terme",
   "ANCIENNETELIASSE": "Ancienneté de la liasse fiscale",
   "INDICATEURETATLIASSE": "Indicateur d'état de la liasse",
   "MTENCOURSNOTE5CONTREPARTIE": "Montant encours note 5 de la contrepartie",
   "MTENCOURSNOTE4CONTREPARTIE": "Montant encours note 4 de la contrepartie",
   "MTENCOURSNOTE3CONTREPARTIE": "Montant encours note 3 de la contrepartie",
   "MTENCOURSNOTE2CONTREPARTIE": "Montant encours note 2 de la contrepartie",
   "MTENCOURSNOTE1CONTREPARTIE": "Montant encours note 1 de la contrepartie",
   "CHIFFREAFFAIRES": "Chiffre d'affaires",
   "EXCEDENTBRUTEXPLOIT": "Excédent brut d'exploitation",
   "TOTALACTIF": "Total actif",
   "FONDSPROPRES": "Fonds propres",
   "MTDEMANDE": "Montant de la demande",
   "CODECANALDISTRIBUTPARTENAIRE": "Code du canal de distribution du partenaire",
   "CODETYPEMODELE": "Code du type de modèle",
   "CODECANALAPPORTPARTENAIRE": "Code du canal d'apport du partenaire",
   "NOTE_LUC": "Note LUC",
   "NOTE_IRPRO": "Note IRPRO",
   "NOTE_NOR": "Note NOR",
   "NOTE_NOFC": "Note NOFC",
   "NOTE_SRR": "Note SRR",
   "CODESEGMENTBALOISCTP": "Code segment bâlois de la contrepartie",
   "SITUATIONRISQPIRECTP": "Situation de risque pire de la contrepartie",
   "CODENAFCTP_5": "Code NAF de la contrepartie à 5 caractères",
   "DATE_CREATION_ROLE": "Date de création du rôle",
   "DATE_CREATION_ROLE_NA": "Date de création du rôle (alternative)",
   "DATE_ARRETE_BDF": "Date d'arrêté de la Banque de France"
}

In [34]:
def check_variables_in_datasets(features_dict, reference_dataset, backtesting_dataset):
    """
    Vérifie la présence des variables spécifiées dans les datasets de référence et de backtesting.
    
    Args:
        features_dict (dict): Dictionnaire des variables avec leurs descriptions
        reference_dataset (pd.DataFrame): Dataset de référence
        backtesting_dataset (pd.DataFrame): Dataset de backtesting
    
    Returns:
        dict: Statistiques sur les variables trouvées/non trouvées dans chaque dataset
    """
    # Colonnes dans chaque dataset
    ref_columns = set(reference_dataset.columns)
    backtest_columns = set(backtesting_dataset.columns)
    
    # Variables à rechercher
    features_to_check = set(features_dict.keys())
    
    # Vérification de la présence dans chaque dataset
    found_in_ref = features_to_check.intersection(ref_columns)
    missing_in_ref = features_to_check - found_in_ref
    
    found_in_backtest = features_to_check.intersection(backtest_columns)
    missing_in_backtest = features_to_check - found_in_backtest
    
    # Variables manquantes dans les deux datasets
    missing_in_both = missing_in_ref.intersection(missing_in_backtest)
    
    # Préparer les statistiques
    stats = {
        "reference_dataset": {
            "total_columns": len(ref_columns),
            "found_variables": len(found_in_ref),
            "missing_variables": len(missing_in_ref),
            "missing_variables_list": sorted(list(missing_in_ref))
        },
        "backtesting_dataset": {
            "total_columns": len(backtest_columns),
            "found_variables": len(found_in_backtest),
            "missing_variables": len(missing_in_backtest),
            "missing_variables_list": sorted(list(missing_in_backtest))
        },
        "both_datasets": {
            "missing_in_both": len(missing_in_both),
            "missing_in_both_list": sorted(list(missing_in_both))
        }
    }
    
    # Afficher un rapport
    print(f"=== RAPPORT DE VÉRIFICATION DES VARIABLES ===")
    print(f"Total des variables à vérifier: {len(features_to_check)}")
    print("\nDataset de référence:")
    print(f"  - Variables trouvées: {stats['reference_dataset']['found_variables']}")
    print(f"  - Variables manquantes: {stats['reference_dataset']['missing_variables']}")
    if stats['reference_dataset']['missing_variables'] > 0:
        print(f"  - Liste des variables manquantes: {stats['reference_dataset']['missing_variables_list']}")
    
    print("\nDataset de backtesting:")
    print(f"  - Variables trouvées: {stats['backtesting_dataset']['found_variables']}")
    print(f"  - Variables manquantes: {stats['backtesting_dataset']['missing_variables']}")
    if stats['backtesting_dataset']['missing_variables'] > 0:
        print(f"  - Liste des variables manquantes: {stats['backtesting_dataset']['missing_variables_list']}")
    
    print("\nVariables manquantes dans les deux datasets:")
    print(f"  - Nombre: {stats['both_datasets']['missing_in_both']}")
    if stats['both_datasets']['missing_in_both'] > 0:
        print(f"  - Liste: {stats['both_datasets']['missing_in_both_list']}")
    
    return stats

In [35]:
stats_bq = check_variables_in_datasets(features_necessaires, ref_sample_bq, bkt_sample_bq)

=== RAPPORT DE VÉRIFICATION DES VARIABLES ===
Total des variables à vérifier: 40

Dataset de référence:
  - Variables trouvées: 29
  - Variables manquantes: 11
  - Liste des variables manquantes: ['DATEIMMATCTP', 'DATE_ARRETE_BDF', 'DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_ARRETE_SRR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']

Dataset de backtesting:
  - Variables trouvées: 32
  - Variables manquantes: 8
  - Liste des variables manquantes: ['DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']

Variables manquantes dans les deux datasets:
  - Nombre: 8
  - Liste: ['DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']


In [36]:
stats_st = check_variables_in_datasets(features_necessaires, ref_sample_st, bkt_sample_st)

=== RAPPORT DE VÉRIFICATION DES VARIABLES ===
Total des variables à vérifier: 40

Dataset de référence:
  - Variables trouvées: 28
  - Variables manquantes: 12
  - Liste des variables manquantes: ['DATEIMMATCTP', 'DATE_ARRETE_BDF', 'DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_ARRETE_SRR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_LUC', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']

Dataset de backtesting:
  - Variables trouvées: 32
  - Variables manquantes: 8
  - Liste des variables manquantes: ['DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']

Variables manquantes dans les deux datasets:
  - Nombre: 8
  - Liste: ['DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']


In [37]:
stats_hst = check_variables_in_datasets(features_necessaires, ref_sample_hst, bkt_sample_hst)

=== RAPPORT DE VÉRIFICATION DES VARIABLES ===
Total des variables à vérifier: 40

Dataset de référence:
  - Variables trouvées: 28
  - Variables manquantes: 12
  - Liste des variables manquantes: ['DATEIMMATCTP', 'DATE_ARRETE_BDF', 'DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_ARRETE_SRR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_LUC', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']

Dataset de backtesting:
  - Variables trouvées: 32
  - Variables manquantes: 8
  - Liste des variables manquantes: ['DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']

Variables manquantes dans les deux datasets:
  - Nombre: 8
  - Liste: ['DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']


Certaines variables on va les prendre du train de base_defaut, comme ça, on aura le même nombre des variables sur rérérence et backtesting.

In [38]:
features_add = [
    'DATEIMMATCTP',
    'DATE_ARRETE_BDF',
    'DATE_ARRETE_SRR',
    'NOTE_LUC',
]

features_add_bq = [
    'DATEIMMATCTP',
    'DATE_ARRETE_BDF',
    'DATE_ARRETE_SRR',    
]

In [39]:
def add_features(ref_df, source_df, features_to_add):
    """
    Ajoute des colonnes à ref_df à partir de source_df en utilisant une jointure gauche pour garder 
    toutes les lignes de ref_sample avec vérification des colonnes et gestion des doublons
    """
    # Vérifier les colonnes existantes
    existing_features = [f for f in features_to_add if f in source_df.columns]
    
    if not existing_features:
        print("Aucune des features à ajouter n'existe dans le DataFrame source")
        return ref_df
    
    # Gérer les doublons dans le DataFrame source
    source_unique = source_df.drop_duplicates(
        subset=['NUMERODOSSIER', 'NUMERODEMANDE'], 
        keep='first'
    )[['NUMERODOSSIER', 'NUMERODEMANDE'] + existing_features]
    
    # Jointure
    ref_df = ref_df.merge(
        source_unique,
        on=['NUMERODOSSIER', 'NUMERODEMANDE'],
        how='left'
    )
    
    return ref_df

# Application
ref_sample_st = add_features(ref_sample_st, base_defaut_train_st, features_add)
ref_sample_hst = add_features(ref_sample_hst, base_defaut_train_hst, features_add)
ref_sample_bq = add_features(ref_sample_bq, base_defaut_train_bq, features_add_bq)

In [40]:
models_data = {
    'BQ': (ref_sample_bq, base_defaut_train_bq),
    'ST': (ref_sample_st, base_defaut_train_st),
    'HST': (ref_sample_hst, base_defaut_train_hst)
}

# Affichage pour chaque modèle
for model_name, (ref_data, bkt_data) in models_data.items():
    print_model_comparison(model_name, ref_data, bkt_data)


COMPARAISON          BQ         
MÉTRIQUE                            |    TRAIN (REF) | TRAIN (BASE DEFAUT)
------------------------------------|----------------|--------------
Taux de cible par échantillon (%)   |           2.55 |           2.53
Volume par échantillon (K)          |          27.39 |          29.15
Part du volume par échantillon (%)  |          48.40 |          51.60
Part de défaut en valeur (cible=1)  |         698.00 |         736.00

COMPARAISON          ST         
MÉTRIQUE                            |    TRAIN (REF) | TRAIN (BASE DEFAUT)
------------------------------------|----------------|--------------
Taux de cible par échantillon (%)   |           3.41 |           3.52
Volume par échantillon (K)          |          31.84 |          33.08
Part du volume par échantillon (%)  |          49.10 |          50.90
Part de défaut en valeur (cible=1)  |        1086.00 |        1163.00

COMPARAISON         HST         
MÉTRIQUE                            |    TRAIN (RE

In [41]:
stats_bq = check_variables_in_datasets(features_necessaires, ref_sample_bq, bkt_sample_bq)

=== RAPPORT DE VÉRIFICATION DES VARIABLES ===
Total des variables à vérifier: 40

Dataset de référence:
  - Variables trouvées: 32
  - Variables manquantes: 8
  - Liste des variables manquantes: ['DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']

Dataset de backtesting:
  - Variables trouvées: 32
  - Variables manquantes: 8
  - Liste des variables manquantes: ['DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']

Variables manquantes dans les deux datasets:
  - Nombre: 8
  - Liste: ['DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']


In [42]:
stats_st = check_variables_in_datasets(features_necessaires, ref_sample_st, bkt_sample_st)

=== RAPPORT DE VÉRIFICATION DES VARIABLES ===
Total des variables à vérifier: 40

Dataset de référence:
  - Variables trouvées: 32
  - Variables manquantes: 8
  - Liste des variables manquantes: ['DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']

Dataset de backtesting:
  - Variables trouvées: 32
  - Variables manquantes: 8
  - Liste des variables manquantes: ['DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']

Variables manquantes dans les deux datasets:
  - Nombre: 8
  - Liste: ['DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']


In [43]:
stats_hst = check_variables_in_datasets(features_necessaires, ref_sample_hst, bkt_sample_hst)

=== RAPPORT DE VÉRIFICATION DES VARIABLES ===
Total des variables à vérifier: 40

Dataset de référence:
  - Variables trouvées: 32
  - Variables manquantes: 8
  - Liste des variables manquantes: ['DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']

Dataset de backtesting:
  - Variables trouvées: 32
  - Variables manquantes: 8
  - Liste des variables manquantes: ['DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']

Variables manquantes dans les deux datasets:
  - Nombre: 8
  - Liste: ['DATE_ARRETE_NOFC', 'DATE_ARRETE_NOR', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_IRPRO', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']


In [44]:
def copy_rename_columns(df, mapping_dict):
    """
    Copie les colonnes avec de nouveaux noms selon le mapping fourni
    sans écraser les colonnes existantes
    
    Args:
        df: DataFrame à modifier
        mapping_dict: Dictionnaire {'nouveau_nom': 'ancien_nom'}
    """
    for new_name, old_name in mapping_dict.items():
        # Vérifier si la colonne source existe
        if old_name in df.columns:
            # Ne créer la nouvelle colonne que si elle n'existe pas déjà
            if new_name not in df.columns:
                df[new_name] = df[old_name]
            else:
                print(f"Attention: La colonne {new_name} existe déjà - non modifiée")
        else:
            print(f"Attention: Colonne source {old_name} non trouvée")
    return df



In [45]:
# Mapping des colonnes pour BQ
mapping_bq = {
    'DATE_ARRETE_NOR': 'DATE_NOTE_NOR_BK',
    'DATE_ARRETE_NOFC': 'DATE_NOTE_NOFC_BK',
    'DATE_CREATION_ROLE': 'D_CREATION_ROLE',
    'DATE_CREATION_ROLE_NA': 'D_CREATION_ROLE_NA',
    'NOTE_NOFC': 'NOTE_NOFC_BK',
    'NOTE_NOR': 'NOTE_NOR_BK',
    'NOTE_SRR': 'NOTE_SRR_PRD',
    'NOTE_IRPRO': 'IRPRO'  # Sera ignoré si IRPRO n'existe pas (en ST et HST)
}

# Mapping pour ST et HST (exclut IRPRO)
mapping_st_hst = {k: v for k, v in mapping_bq.items() if v != 'IRPRO'}

# Application aux DataFrames
ref_sample_bq = copy_rename_columns(ref_sample_bq, mapping_bq)
ref_sample_st = copy_rename_columns(ref_sample_st, mapping_st_hst)
ref_sample_hst = copy_rename_columns(ref_sample_hst, mapping_st_hst)

# Vérification
print("Colonnes ajoutées dans ref_sample_bq:", [k for k in mapping_bq if k in ref_sample_bq.columns])
print("Colonnes ajoutées dans ref_sample_st:", [k for k in mapping_st_hst if k in ref_sample_st.columns])

Colonnes ajoutées dans ref_sample_bq: ['DATE_ARRETE_NOR', 'DATE_ARRETE_NOFC', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR', 'NOTE_IRPRO']
Colonnes ajoutées dans ref_sample_st: ['DATE_ARRETE_NOR', 'DATE_ARRETE_NOFC', 'DATE_CREATION_ROLE', 'DATE_CREATION_ROLE_NA', 'NOTE_NOFC', 'NOTE_NOR', 'NOTE_SRR']


In [46]:
bkt_sample_bq = copy_rename_columns(bkt_sample_bq, mapping_bq)
bkt_sample_st = copy_rename_columns(bkt_sample_st, mapping_st_hst)
bkt_sample_hst = copy_rename_columns(bkt_sample_hst, mapping_st_hst)

In [47]:
stats_bq = check_variables_in_datasets(features_necessaires, ref_sample_bq, bkt_sample_bq)

=== RAPPORT DE VÉRIFICATION DES VARIABLES ===
Total des variables à vérifier: 40

Dataset de référence:
  - Variables trouvées: 40
  - Variables manquantes: 0

Dataset de backtesting:
  - Variables trouvées: 40
  - Variables manquantes: 0

Variables manquantes dans les deux datasets:
  - Nombre: 0


In [48]:
stats_st = check_variables_in_datasets(features_necessaires, ref_sample_st, bkt_sample_st)

=== RAPPORT DE VÉRIFICATION DES VARIABLES ===
Total des variables à vérifier: 40

Dataset de référence:
  - Variables trouvées: 39
  - Variables manquantes: 1
  - Liste des variables manquantes: ['NOTE_IRPRO']

Dataset de backtesting:
  - Variables trouvées: 39
  - Variables manquantes: 1
  - Liste des variables manquantes: ['NOTE_IRPRO']

Variables manquantes dans les deux datasets:
  - Nombre: 1
  - Liste: ['NOTE_IRPRO']


In [49]:
stats_hst = check_variables_in_datasets(features_necessaires, ref_sample_hst, bkt_sample_hst)

=== RAPPORT DE VÉRIFICATION DES VARIABLES ===
Total des variables à vérifier: 40

Dataset de référence:
  - Variables trouvées: 39
  - Variables manquantes: 1
  - Liste des variables manquantes: ['NOTE_IRPRO']

Dataset de backtesting:
  - Variables trouvées: 39
  - Variables manquantes: 1
  - Liste des variables manquantes: ['NOTE_IRPRO']

Variables manquantes dans les deux datasets:
  - Nombre: 1
  - Liste: ['NOTE_IRPRO']


La note IRPRO est spécifique au canal bancaire, notament pour LCL. Dans le data_cleaner on peut voir que la note IRPRO est attribuée uniquement lorsque le canal d'apport du partenaire est LCL:

def attribute_luc_irpro(X: DataFrame):

    # attribution IRPRO ou LUC

    note_bal = nan

    if (X["CODECANALDISTRIBUTPARTENAIRE"] == "BANQ") & (
        X["CODETYPEMODELE"] == "GLES"
    ):

        note_bal = X["NOTE_LUC"]

    elif X["CODECANALAPPORTPARTENAIRE"] == "LCL":
    
        note_bal = X["NOTE_IRPRO"]


#### 2. Gestion des colonnes manquantes

Je constate qu'il ne contient pas de gestion explicite des valeurs manquantes pour les colonnes qui pourraient être absentes du dataframe. Il s'attend à ce que toutes les colonnes nécessaires soient présentes lors de l'exécution.
Voici ce qui manque dans la version de production:

- Il n'y a pas de vérification des colonnes manquantes
- Il n'y a pas d'initialisation des colonnes absentes à NaN
- Il n'y a pas de gestion des erreurs si une colonne requise n'existe pas



In [ ]:
# # Dans la fonction de patch qu'on va faire, on va prendre en compte les variables manquantes
# # à mettre dans le pipeline de nettoyage
# missing_var = [c for c in self.features if c not in X.columns]
# df[missing_var] = np.nan

### 2. Correction des valeurs manquantes pour les variables d'évolution

In [51]:
def replace_999_values(df, special_value=-999, nb_bilans_col='NBBILANS', nb_bilans_value=1):
    """
    Remplace les valeurs spéciales (-999 par défaut) par NaN pour les entreprises avec un seul bilan.
    
    Args:
        df: DataFrame à traiter
        special_value: Valeur à remplacer (par défaut -999)
        nb_bilans_col: Nom de la colonne indiquant le nombre de bilans (par défaut 'NBBILANS')
        nb_bilans_value: Valeur seuil pour le nombre de bilans (par défaut 1)
    
    Returns:
        DataFrame modifié
    """
    cols_to_process = [
        "EVOLFONDSPROPRESSURBILAN", 
        "EVOLRESULTATNETANNUALISE",
        "EVOLTRESORERIESURJCAANNUEL", 
        "EVOLCA"
    ]
    
    for col in cols_to_process:
        if col in df.columns:
            mask = (df[col] == special_value) & (df[nb_bilans_col] == nb_bilans_value)
            df.loc[mask, col] = np.nan
    
    return df

In [52]:
ref_sample_bq = replace_999_values(ref_sample_bq)
ref_sample_st = replace_999_values(ref_sample_st)
ref_sample_hst = replace_999_values(ref_sample_hst)

In [53]:
bkt_sample_bq = replace_999_values(bkt_sample_bq)
bkt_sample_st = replace_999_values(bkt_sample_st)
bkt_sample_hst = replace_999_values(bkt_sample_hst)

In [ ]:
def patch_before_data_cleaner(df):
    """
    Patch qui crée les colonnes intermédiaires attendues par data_cleaner
    à partir des colonnes finales déjà présentes dans le dataframe.
    """
    # # Mapping entre les noms finaux et intermédiaires
    # mapping = {
    #     "ANCIENNETELIASSE": "DIFDATECLOTUREBILAN",
    #     "ANCIENNETECONTREPARTIE": "DIFDATEIMMATCTP",
    #     "ANCIENNETENOR": "DIFDATE_ARRETE_NOR",
    #     "ANCIENNETENOFC": "DIFDATE_ARRETE_NOFC",
    #     "ANCIENNETESRR": "DIFDATE_ARRETE_SRR"
    # }
    
    # Copie du dataframe pour éviter les modifications sur l'original
    df = df.copy()

    # 1. Gestion des colonnes manquantes
    missing_var = [c for c in self.features if c not in X.columns]
    df[missing_var] = np.nan

    # 2. Correction des valeurs manquantes pour les variables d'évolution
    for col in ["EVOLFONDSPROPRESSURBILAN", "EVOLRESULTATNETANNUALISE", 
                "EVOLTRESORERIESURJCAANNUEL", "EVOLCA"]:
        if col in df.columns:
            mask = (df[col] == -999) & (df["NBBILANS"] == 1)
            df.loc[mask, col] = np.nan

    # 3. Convertir la note bien prinicipal en variable categorielle
    df['NOTEBIENPRINCIPAL'] = df['NOTEBIENPRINCIPAL'].astype(str)
    
    # 4. Renommage des colonnes selon les correspondances
    column_mapping = {
        "ANCIENNETECONTREPARTIE": "DIFDATEIMMATCTP",
        "ANCIENNETE_NOTEANADEFINOR": "DIFDATE_ARRETE_NOR",
        "ANCIENNETE_NOTEANADEFINOFC": "DIFDATE_ARRETE_NOFC",
        "ANCIENNETE_NOTESRR": "DIFDATE_ARRETE_SRR",
        "IRPRO": "NOTE_IRPRO",
        "NOTE_SRR_PRD": "NOTE_SRR",
        "NOTE_NOFC_BK": "NOTE_NOFC",
        "D_CREATION_ROLE": "DATE_CREATION_ROLE",
        "D_CREATION_ROLE_NA": "DATE_CREATION_ROLE_NA",
        "NOTE_NOR_BK": "NOTE_NOR",
    }
    df = df.rename(columns={v: k for k, v in column_mapping.items() if v in df.columns})
    
    # 2. Conversion de la datedemande au format datetime
    df["DATEDEMANDE"] = pd.to_datetime(df["DATEDEMANDE"])
    df["DATE_ARRETE_BDF"] = pd.to_datetime(df["DATE_ARRETE_BDF"])
    
    # 3. Convertir la NOTEBIENPRINCIPAL en string
    if "NOTEBIENPRINCIPAL" in df.columns:
        df["NOTEBIENPRINCIPAL"] = df["NOTEBIENPRINCIPAL"].astype(str)
    
    
    
    # 5. Correction des valeurs négatives
    for col in ["MTIMP3MOISCTP", "ENCOURSMAX24MOISGROUPE"]:
        if col in df.columns:
            mask = df[col] < 0
            df.loc[mask, col] = 0
    
    # 6. Correction du montant unitaire du bien principal
    if all(col in df.columns for col in ["MONTANTUNITAIREBIENPRINCIPAL", "MTDEMANDE"]):
        mask = df["MONTANTUNITAIREBIENPRINCIPAL"] > df["MTDEMANDE"]
        df.loc[mask, "MONTANTUNITAIREBIENPRINCIPAL"] = df.loc[mask, "MTDEMANDE"]
        
        mask = df["MONTANTUNITAIREBIENPRINCIPAL"] == 0
        df.loc[mask, "MONTANTUNITAIREBIENPRINCIPAL"] = df.loc[mask, "MTDEMANDE"]
    
    # 7. Création des colonnes intermédiaires attendues par le data_cleaner
    # Ces colonnes sont calculées dans le data_cleaner mais nécessaires
    mapping = {
        "ANCIENNETELIASSE": "DIFDATECLOTUREBILAN",
        "ANCIENNETECONTREPARTIE": "DIFDATEIMMATCTP"
    }
    for final_col, intermediate_col in mapping.items():
        if final_col in df.columns:
            df[intermediate_col] = df[final_col]
    
    # 8. Construction de la variable CODENAFCTP_EXPERT_2 si nécessaire
    # Sera recalculée par le data_cleaner mais peut être nécessaire pour d'autres calculs
    if "CODENAFCTP_5" in df.columns and "CODENAFCTP_EXPERT_2" not in df.columns:
        # Le dictionnaire naf_expert_2 est défini dans data_cleaner
        # Cette ligne est optionnelle car sera recalculée par le data_cleaner
        pass
    
    # 9. Construction de la catégorie experte pour les biens
    if "CODENATUREBIENPRINCIPAL" in df.columns:
        nature_codes = ["R19P", "R20P", "R23S", "R20W", "R20R", "R20S", "R20T", "R20U",
                       "R20Q", "R16P", "R19Q", "R21S", "R22S", "R24S", "R23S", "R20W", 
                       "R25S", "R26S", "R27S", "R19Q"]
        
        df["CODECATEGORIEBIENPRINCIPAL_EXPERT"] = df["CODECATEGORIEBIENPRINCIPAL"]
        df["CODENATUREBIENPRINCIPAL_EXPERT"] = df["CODENATUREBIENPRINCIPAL"]
        
        mask = df["CODENATUREBIENPRINCIPAL"].isin(nature_codes)
        df.loc[mask, "CODECATEGORIEBIENPRINCIPAL_EXPERT"] = "R1920P_expert"
        df.loc[mask, "CODENATUREBIENPRINCIPAL_EXPERT"] = "R1920P_expert"
    
    # 10. Ajout des ratios supplémentaires
    ratios = [
        ("DETTESINF1AN", "CAPACITEAUTOFINANCEMENT", "DETTESINF1AN_CAPACITEAUTOFINANCEMENT"),
        ("BESOINFONDSDEROULEMENT", "TRESORERIE", "BESOINFONDSDEROULEMENT_TRESORERIE"),
        ("COURTTERME", "TRESORERIE", "COURTTERME_TRESORERIE"),
        ("EVOLCA", "BESOINFONDSDEROULEMENT", "EVOLCA_BESOINFONDSDEROULEMENT"),
        ("BESOINFONDSDEROULEMENT", "FONDSDEROULT", "BESOINFONDSDEROULEMENT_FONDSDEROULT")
    ]
    
    for num, denom, result in ratios:
        if all(col in df.columns for col in [num, denom]):
            df[result] = df[num] / df[denom].replace(0, np.nan)
    
    # 11. Calcul du ratio TOTALCOURTTERME_TOTALACTIF
    if all(col in df.columns for col in ["COURTTERME", "AUTRESCOURTTERME", "TOTALACTIF"]):
        df["TOTALCOURTTERME_TOTALACTIF"] = (df["COURTTERME"] + df["AUTRESCOURTTERME"]) / df["TOTALACTIF"].replace(0, np.nan)
    
    # 12. Calcul et correction de MTENCOURSNOTE5CONTREPARTIE_MTENCOURS
    mtencours = [
        "MTENCOURSNOTE5CONTREPARTIE",
        "MTENCOURSNOTE4CONTREPARTIE",
        "MTENCOURSNOTE3CONTREPARTIE",
        "MTENCOURSNOTE2CONTREPARTIE",
        "MTENCOURSNOTE1CONTREPARTIE",
    ]
    
    if all(col in df.columns for col in mtencours):
        df["MTENCOURS"] = df[mtencours].sum(axis=1)
        df["MTENCOURSNOTE5CONTREPARTIE_MTENCOURS"] = (df["MTENCOURSNOTE5CONTREPARTIE"] / df["MTENCOURS"].replace(0, np.nan)).fillna(0)
    
    # 13. Correction des valeurs aberrantes pour MTENCOURSFUTUR_FONDSPROPRES
    if "MTENCOURSFUTUR_FONDSPROPRES" in df.columns:
        mask = df["MTENCOURSFUTUR_FONDSPROPRES"] > 9.312703
        df.loc[mask, "MTENCOURSFUTUR_FONDSPROPRES"] = 0
        
        mask = df["MTENCOURSFUTUR_FONDSPROPRES"] < -0.009010
        df.loc[mask, "MTENCOURSFUTUR_FONDSPROPRES"] = 0
    
    # 14. Correction des valeurs aberrantes pour ANCIENNETECONTREPARTIE
    if "ANCIENNETECONTREPARTIE" in df.columns:
        mask = df["ANCIENNETECONTREPARTIE"] > 769
        df.loc[mask, "ANCIENNETECONTREPARTIE"] = np.nan


    return df
    



# Utilisation
# df = patch_before_data_cleaner(df)
# Puis appliquer le data_cleaner

In [ ]:
base_defaut_bq_processed= bq_pipeline.fit_transform(base_defaut_bq)
base_defaut_st_processed= st_pipeline.fit_transform(base_defaut_st)
base_defaut_hst_processed= hst_pipeline.fit_transform(base_defaut_hst)

KeyError: 'DATE_ARRETE_NOR'

### Anomalie COTECREDIT

Cette variable est construite à partir du NOTEBDFCTP et elle represente le 2ème caractère des valeurs de NOTEBDFCTP, sauf "X0" ou COTECREDIT prend aussi la valeur X0.

D'après le code disponible dans data_cleaner.py, la construction se fait ainsi:

*construction cotecredit*

X["COTECREDIT"] = X["NOTEBDFCTP"].apply(
    lambda u: u[1:] if notna(u) else u
)

*différenciation entre les X0 et les autres 0*

X.loc[X["NOTEBDFCTP"] == "X0", "COTECREDIT"] = "X0"

## Analyse scorer